In [39]:
import albumentations as A
import cv2
import glob
import numpy as np
import random
from matplotlib import pyplot as plt
#path=D:/FDM_DOWNLOAD/test/test/images/7202.jpg


def Augmentation(path):
# Load image and CSV  
    images = glob.glob(path)
    df = pd.read_csv('train/train.csv')
    filename = os.path.basename(images[0])
    #print(filename)
    images = cv2.imread(images[0])
    image = np.array(images)
    
   


    # Define transforms
    transforms = A.Compose([
        A.Resize(width=512, height=512), # Scale all images to 256x256

        A.Flip(p=0.5), # Random flip half the images horizontally
        A.Transpose(p=0.5), # Random transpose half the images
        A.RandomBrightnessContrast(p=0.5), 
        A.MedianBlur(blur_limit=3, p=0.1),
        A.MotionBlur(p=0.2),
        A.GaussNoise(p=0.1),
        A.OpticalDistortion(p=0.3),
        A.GridDistortion(p=0.1),
        A.ElasticTransform(p=0.1, alpha=120, sigma=120 * 0.05, alpha_affine=120 * 0.03),
        A.HueSaturationValue(p=0.5),
        A.CLAHE(p=0.5),
        A.ShiftScaleRotate(shift_limit=0.0625, scale_limit=0.2, rotate_limit=45, p=0.5),
        A.CoarseDropout(p=0.1)  
    ])


    # Get label from CSV
    matches = df.loc[df['filename'] == filename]
    if len(matches) > 0:
       label = matches.iloc[0]['label']
    else:
       print("No match found")
    #print(label)

    output_dir = 'D:/DATA/augmented_images'
    os.makedirs(output_dir, exist_ok=True)
    aug_images = []

    for i in range(15):

        # Augment image
        aug_img = transforms(image=image)['image']
        aug_images.append(aug_img)

        # File name for augmented image
        aug_file = f'{filename}_aug{i}.jpg'
        #print(aug_file)

        # Save image 
        cv2.imwrite(os.path.join(output_dir, aug_file), aug_img)
        new_row = {'filename': aug_file, 'label': label}
        

        df = pd.concat([df, pd.DataFrame([new_row])], ignore_index=True)
        # Append CSV row 



    # Save CSV
    df.to_csv('updated_labels_train.csv', index=False)

    #print(filename,'Done!')

In [37]:
Augmentation('C:/Users/Sri Ram/Untitled Folder 8/train/images/3.jpg')

In [41]:
folder = 'C:/Users/Sri Ram/Untitled Folder 8/train/images'  

import os
from tqdm import tqdm
from PIL import Image
#import augmentations # custom augmentations



image_paths = []
for root, _, files in os.walk(folder):
        for file in tqdm(files): 
            if file.endswith((".jpg", ".jpeg", ".png")):
                image_path = os.path.join(root, file)
                Augmentation(image_path)
                
                pass


100%|██████████████████████████████████████████████████████████████████████████████| 7200/7200 [53:01<00:00,  2.26it/s]


In [43]:
import re

# Folder containing images 
img_dir = 'D:/DATA/augmented_images'

for filename in os.listdir(img_dir):
  # Split filename & extension
  name, ext = os.path.splitext(filename) 
  
  # Extract ID and aug number   
  id_ = re.search(r'^(\d+)', name).group(1)
  aug = re.search(r'_aug(\d+)', name).group(1)
  
  # Construct new name
  new_name = f"{id_}_{aug}{ext}"
  
  # Rename file
  old_path = os.path.join(img_dir, filename)
  new_path = os.path.join(img_dir, new_name)
  os.rename(old_path, new_path)
  
print("Renamed all files successfully")

Renamed all files successfully


In [73]:
img_dir = 'D:/DATA/augmented_images'
filenames = os.listdir(img_dir)
csv_path = 'train/train.csv'
labels_df = pd.read_csv(csv_path)
# Extract ID from filename
def get_id(fname):
  return re.search(r'^(\d+)_', fname).group(1)

# Map filenames to labels  
file_labels = [] 
for fname in filenames:
  id_ = get_id(fname)  
  label = labels_df[labels_df['image_id']==int(id_)]['label'].values[0]
  file_labels.append({'id':id_,'filename': fname, 'label': label})
  
# Generate output CSV  
output_df = pd.DataFrame(file_labels)
output_df.to_csv('output.csv', index=False) 

print('CSV saved')


CSV saved


In [75]:
df["label"].valuecounts()

AttributeError: 'Series' object has no attribute 'valuecounts'